# 3D Cylinder
***
This is an example of a 3D reaction diffusion system.
- The domain is a cylinder
- Species A is created at the right end of the cylinder
- Species B is created at the left end of the cylinder
- Diffusion occurs throughout the volume of the cylinder
- A and B are destroyed on contact
- Deterministic and Stochastic simulations are run

This example allows us to check if reaction diffusion dynamics are accurate.
***
## Setup the Environment
***

In [ ]:
import spatialpy

***
## Geometries
***

In [ ]:
class Edge1(spatialpy.Geometry):
    def __init__(self):
        pass

    def inside(self, point, on_boundary):
        namespace = {'x': point[0], 'y': point[1], 'z': point[2]}
        formula = 'abs(x - 5) < 0.05'
        return eval(formula, {}, namespace)

In [ ]:
class Edge2(spatialpy.Geometry):
    def __init__(self):
        pass

    def inside(self, point, on_boundary):
        namespace = {'x': point[0], 'y': point[1], 'z': point[2]}
        formula = 'abs(x + 5) < 0.05'
        return eval(formula, {}, namespace)

***
## Create the 3D Cylinder Model
***

In [ ]:
def create_3dcylinder(parameter_values=None):
    model = spatialpy.Model(name='3D-Cylinder')

    # Define Domain Type IDs as constants of the Model
    model.EDGE1 = 'Edge1'
    model.EDGE2 = 'Edge2'
    model.MIDDLE = 'Middle'

    # Domain
    domain = spatialpy.Domain(
        0, (-5, 5), (-1, 1),
        (-1, 1), rho0=1, c0=10, P0=100, gravity=None
    )

    # Domain Geometries
    edge1_geom = Edge1()
    edge2_geom = Edge2()
    middle_geom = spatialpy.CombinatoryGeometry('', {})
    middle_geom.formula = 'not (edge1 or edge2)'

    middle_geom.geo_namespace = {'edge1': edge1_geom, 'edge2': edge2_geom}

    import os
    home = os.path.expanduser('~')
    # Domain Lattices
    ipa_lattice1 = spatialpy.XMLMeshLattice(
        os.path.join(home, 'Examples/3D-Cylinder.proj/Domain Files/cylinder.xml')
    )

    # Domain Actions
    domain.add_fill_action(
        lattice=ipa_lattice1, enable=True, apply_action=True,
        mass=1, vol=1, rho=1, nu=0, c=10, fixed=False
    )
    domain.add_set_action(
        geometry=middle_geom, enable=True, apply_action=True,
        type_id=model.MIDDLE, mass=1, vol=1, rho=1, nu=0, c=10, fixed=False
    )
    domain.add_set_action(
        geometry=edge1_geom, enable=True, apply_action=True,
        type_id=model.EDGE1, mass=1, vol=1, rho=1, nu=0, c=10, fixed=False
    )
    domain.add_set_action(
        geometry=edge2_geom, enable=True, apply_action=True,
        type_id=model.EDGE2, mass=1, vol=1, rho=1, nu=0, c=10, fixed=False
    )

    model.add_domain(domain, allow_all_types=True)

    model.staticDomain = True

    # Variables
    A = spatialpy.Species(name='A', diffusion_coefficient=0.1, restrict_to=[model.EDGE1, model.MIDDLE])
    B = spatialpy.Species(name='B', diffusion_coefficient=0.1, restrict_to=[model.EDGE2, model.MIDDLE])
    model.add_species([
        A, B
    ])

    # Parameters
    left = spatialpy.Parameter(name='left', expression='0.5092013833059308')
    right = spatialpy.Parameter(name='right', expression='0.505804729089437')
    k_react = spatialpy.Parameter(name='k_react', expression='1')
    k_creat1 = spatialpy.Parameter(name='k_creat1', expression='100')
    k_creat2 = spatialpy.Parameter(name='k_creat2', expression='100')
    model.add_parameter([
        left, right, k_react, k_creat1, k_creat2
    ])

    # Reactions
    r1 = spatialpy.Reaction(
        name='r1', restrict_to=[model.EDGE1],
        reactants={}, products={'A': 1},
        propensity_function='k_creat1 / left * vol',
        ode_propensity_function='k_creat1'
    )
    r2 = spatialpy.Reaction(
        name='r2', restrict_to=[model.EDGE2],
        reactants={}, products={'B': 1},
        propensity_function='k_creat2 / right * vol',
        ode_propensity_function='k_creat2'
    )
    r3 = spatialpy.Reaction(
        name='r3', rate='k_react', restrict_to=[model.MIDDLE, model.EDGE2, model.EDGE1],
        reactants={'A': 1, 'B': 1}, products={}
    )
    model.add_reaction([
        r1, r2, r3
    ])

    # Timespan
    tspan = spatialpy.TimeSpan.arange(1, t=500, timestep_size=1)
    model.timespan(tspan)
    return model

### Instantiate the model

In [ ]:
model = create_3dcylinder()

In [ ]:
model.domain

***
## Simulation Parameters
***

In [ ]:
def configure_simulation():
    kwargs = {
        # 'number_of_trajectories': 1,
        # 'seed': None,
    }
    return kwargs

***
## Run the Simulation
***

In [ ]:
kwargs = configure_simulation()
results = model.run(**kwargs)

***
## Visualization
***

In [ ]:
results.plot_species('A', animated=True, width='auto', height='auto')